In [ ]:
from datasets import load_dataset

dataset = load_dataset("agkphysics/AudioSet", streaming=True)

Loading dataset shards:   0%|          | 0/47 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
for example in dataset:
    print(example['human_labels'])

In [ ]:

print(dataset['train'][0]['audio'])

{'path': 'audio/bal_train/--PJHxphWEs.flac', 'array': array([-0.04364824, -0.05268681, -0.0568949 , ...,  0.11446512,
        0.14912748,  0.13409865]), 'sampling_rate': 48000}


In [7]:
print(dataset['train'][3]['human_labels'])
print(dataset['train'][3]['labels'])

['Gunshot, gunfire', 'Cap gun']
['/m/032s66', '/m/073cg4']


In [3]:
print(dataset['train'][0]['human_labels'])
print(dataset['train'][0]['labels'])

for example in dataset['train']:
    s = set()
    for label in example['labels']:
        s.add(label)
    if len(s) != len(example['labels']):
        print(example['labels'])

KeyboardInterrupt: 